# **Horses and Humans Classifier**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

To implement **CNN (Convolution Neural Network)** $\rightarrow$ Use **convolutional** layers in the model definition and also add pooling layers.

Convolution layer with 2D filters: `tf.keras.layers.Conv2D`

This accepts as parameters:
- The number of convolutions to use in the layer
- The size of convolutions
- The activation function
- ...

In [10]:
tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1))

c:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<Conv2D name=conv2d_1, built=False>

In this case, we want the layer to learn `64` convolutions. 
- It will **randomly initialize** these, and over time will learn **the filter values** that **work best** to match the input values to their labels.
- The **(3, 3)** indicates **the size of the filter**. This is the most common size of filter. But you'll typically see an **odd** number of axes like 5x5 or 7x7 because of how filters remove pixels from the borders of the image.
- Because **Conv2D** layers are designed for **multicolor images**, we're specifying the third dimension as 3 because the color images will typically have a 3 as third parameter as they are stored as values of R, G and B.

Use a `pooling` layer in **CNN** -> do this immediately after the convolution layer.

In [11]:
tf.keras.layers.MaxPooling2D(2, 2)

<MaxPooling2D name=max_pooling2d, built=True>

### **Load the dataset**

In [31]:
training_dir = 'data/horse-or-human/train/'
validation_dir = 'data/horse-or-human/validation/'

# All images (training and validation images) will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary',
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode='binary',
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


### **CNN Architecture for Horses or Humans**

**Note**

1. The images are much larger - 300 x 300 pixels - so more layers may be needed.

$\quad \Rightarrow$ Stack several more convolution layers and we want, over time, to have many smaller images, each with features highlighted.

2. The images are full color (not grayscale anymore) so each image will have three channels instead of one.
3. There are only two image types, so we have a binary classifier that can be implemented using just a single output neuron, where it approaches 0 for 1 class and 1 for the other. 

**The model architecture**

In [25]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation=tf.nn.relu, input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 298, 298, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 149, 149, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 147, 147, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 33, 33, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,606,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,704,097 (6.50 MB)

 Trainable params: 1,704,097 (6.50 MB)

 Non-trainable params: 0 (0.00 B)

To train => have to compile it with a **loss function** and an **optimizer**.

In this case, the **loss function** can be **binary cross entropy** because there're only 2 classes, and as the name suggests, this is a loss function that is designed for that scenario.

Try a new optimizer, **root mean square propagation (RMSprop)**, that takes a learning rate (**lr**) parameter that allow us to tweak the learning.

In [26]:
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
history = model.fit(train_generator, epochs=15)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5970 - loss: 2.3023
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 438ms/step - accuracy: 0.8204 - loss: 0.6124
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 455ms/step - accuracy: 0.9020 - loss: 0.2363
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 459ms/step - accuracy: 0.9706 - loss: 0.1046
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 507ms/step - accuracy: 0.9176 - loss: 0.2633
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 483ms/step - accuracy: 0.9457 - loss: 0.1686
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 507ms/step - accuracy: 0.9822 - loss: 0.0354
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 513ms/step - accuracy: 0.9860 - loss: 0.0394
Epoch 9/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 518ms/step - accuracy: 1.0000 - loss: 0.0038
Epoch 10/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 506ms/step - accuracy: 1.0000 - loss: 5.9907e-04
Epoch 11/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 502ms/step - accuracy: 1.0000 - loss: 4.5964e-05
Epoch 12/15
33/33 ━━━━━━━━━━━━━━━━━━

### **Adding Validation to the Horses or Humans Dataset**

**Why is about a validation dataset here, rather than a test dataset, and whether they're the same thing?**

**For simple models**, it's often sufficient to split the dataset into two parts, one for training and one for testing.

But **for more complex models**, you'll want to create separate validation and test sets.

**What's the difference?**
- ***Training data*** is the data that is used to teach the network how the data and labels fit together.
- ***Validation data*** is used to see how the network is doing with previously unseen data while you are training. - i.e., it is not used for fitting data to labels, but to inspect how well the fitting is going.
- ***Test data*** is used after training to see how the network does with data it has never previously seen.

Some datasets come with the three-way split, and in other cases, to separate the test set into 2 parts for validation and testing.

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=15)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 647ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.8281 - val_loss: 2.0611
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 460ms/step - accuracy: 1.0000 - loss: 1.5209e-04 - val_accuracy: 0.8320 - val_loss: 2.5075
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 565ms/step - accuracy: 1.0000 - loss: 3.1599e-05 - val_accuracy: 0.8203 - val_loss: 2.8709
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 576ms/step - accuracy: 1.0000 - loss: 1.2278e-05 - val_accuracy: 0.8203 - val_loss: 3.1458
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 20s 591ms/step - accuracy: 1.0000 - loss: 6.0251e-06 - val_accuracy: 0.8242 - val_loss: 3.4177
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 589ms/step - accuracy: 1.0000 - loss: 8.2654e-06 - val_accuracy: 0.8281 - val_loss: 3.2476
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 580ms/step - accuracy: 1.0000 - loss: 3.1130e-06 - val_accuracy: 0.8242 - val_loss: 3.4992
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 645ms/step - accuracy: 1.0000 -

### **Visualize the training result**

In [ ]:
# Extract history data
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

# Plot Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()